# Filters

This section includes information about filters, which can be a very painful process and somewhat obtuse.  

Filters are basically anything that manipulates the time series data into something else.  This includes scaling, removing instrument response, removing trends, notch filtering, passband filtering, etc.  We want to work with data that is in physical units, but the data are often collected in digital counts.  To transform between the two we need to apply or unapply certain filters.  When going from physical units to digital counts we are applying filters, `channel_metadata.filter.applied = [True]`, whereas going from counts to physical units we are unapplying the filters `channel_metadata.filter.applied = [False]`.  This may seem backwards, but this is the way most archived data is thought of. Then any filter applied to physical units for the purposes of cleaning the data are `channel_metadata.filter.applied = [True]`.     

**Note**: currently there are not tools to convert digital counts to physical units, this is a work in progress.  Nevertheless all the information is there for you to do the transformation.  

Supported filters in `mt_metadata.timeseries.filters` are: 

| Filter | Description |
|--------|-------------|
| CoefficientFilter | A coefficient filter scales the data by a given factor. |
| FIRFilter | | 
| PoleZeroFilter| |
| TimeDelayFilter | | 
| FrequencyResponseTableFilter | |
| ChannelResponseFilter | |
